In [1]:
import json
import nltk
import string
import re
import numpy as np

In [2]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from sklearn.model_selection import train_test_split
# from embed import data_preprocess,get_bert_embeddings

In [3]:
proxy_dataset = {
    "Users are reporting issues with the application's login process. When attempting to log in with their credentials, the system fails to authenticate and displays an error message. This problem seems to occur intermittently across different devices and browsers. The login button appears unresponsive in some instances, preventing users from accessing their accounts. We need to investigate and resolve this issue promptly to ensure seamless user experience and maintain customer satisfaction.": 
    "Check the authentication server logs and ensure the server is running. Restart the server if necessary. Verify that the login endpoint is properly configured and not experiencing downtime. Test the login process across various devices and browsers to identify any specific issues. Consider implementing a backup authentication server to handle login requests if the primary server fails.",

    "Users are experiencing delays when trying to upload files to the application. The upload process takes an unusually long time, and in some cases, it fails completely without any error message. This issue is affecting productivity as users rely on file uploads for their daily tasks.": 
    "Inspect the server load and bandwidth usage to identify any bottlenecks. Ensure the file upload service is optimized for handling large files. Check for any network issues that might be causing delays. Implement error handling to provide feedback to users when uploads fail. Consider using a content delivery network (CDN) to improve upload speeds.",

    "The application's search functionality is returning incomplete or incorrect results. Users are unable to find the information they need, which is impacting their ability to perform their tasks effectively. This issue seems to be related to the indexing of the search data.": 
    "Verify that the search indexing service is running correctly and is up-to-date. Rebuild the search index to ensure all data is accurately indexed. Check for any errors in the search algorithm that might be causing incorrect results. Enhance the search functionality by implementing advanced search filters and options.",

    "Customers are complaining about receiving duplicate email notifications from the system. This issue is causing confusion and frustration among users as they receive multiple emails for the same event or action.": 
    "Review the email notification system to identify any configuration errors. Ensure that duplicate notifications are not being triggered by multiple events. Implement checks to prevent the same notification from being sent more than once. Monitor the email queue and logs to detect and resolve any issues promptly.",

    "The application's user interface is not displaying correctly on certain mobile devices. Some elements are misaligned, and the layout appears broken, making it difficult for users to navigate and use the application effectively.": 
    "Test the application's user interface on various mobile devices to identify specific issues. Adjust the CSS and responsive design settings to ensure compatibility with different screen sizes. Fix any misaligned elements and broken layouts. Implement a mobile-first design approach to improve usability on mobile devices.",

    "Users are reporting that the application crashes unexpectedly during usage. This issue occurs sporadically and does not follow a predictable pattern, making it difficult to identify the root cause.": 
    "Collect and analyze crash logs to identify any common patterns or errors. Ensure that all dependencies and libraries are up-to-date and compatible. Implement additional logging and monitoring to capture detailed information about the crashes. Perform thorough testing to replicate and resolve the issue.",

    "The application's performance has degraded significantly during peak usage times. Users experience slow response times and lag, which affects their ability to use the application efficiently.": 
    "Evaluate the application's performance metrics to identify bottlenecks. Optimize the database queries and server configuration to handle higher loads. Implement load balancing to distribute traffic evenly across servers. Consider scaling up the infrastructure to accommodate peak usage times.",

    "Users are unable to reset their passwords using the password recovery feature. The system does not send the password reset email, leaving users unable to regain access to their accounts.": 
    "Verify that the email service is configured correctly and is operational. Check the logs for any errors related to the password reset process. Ensure that the password reset tokens are being generated and sent correctly. Provide users with alternative methods for password recovery if needed.",

    "The application's API is returning errors when third-party services attempt to connect. This issue is preventing integrations with other systems and affecting functionality that relies on external data.": 
    "Review the API documentation and ensure that all endpoints are correctly implemented. Check the API logs for any errors or issues. Verify that the API keys and authentication methods are working as expected. Test the API integration with third-party services to identify and resolve any compatibility issues.",

    "Users are experiencing issues with the application's payment processing. Transactions are failing, and users are unable to complete their purchases, leading to lost sales and revenue.": 
    "Inspect the payment gateway configuration to ensure it is set up correctly. Check the transaction logs for any errors or failed attempts. Verify that the payment gateway is not experiencing downtime or technical issues. Implement fallback payment options to ensure users can complete their purchases."
}
test_dataset = {
    "Users are facing issues with the application's logout functionality. When attempting to log out, the session does not terminate properly, and users remain logged in. This problem compromises account security as users cannot ensure they have logged out correctly.":
    "Check the session management and ensure that the logout endpoint is terminating sessions correctly. Verify that cookies and tokens are being invalidated upon logout. Review the logs for any errors during the logout process. Implement automatic session timeout as an additional security measure.",

    "The application's dashboard is not loading for some users. When trying to access the dashboard, users see a loading spinner that never completes. This issue affects their ability to view critical data and perform necessary tasks.":
    "Inspect the server logs to identify any errors during the dashboard loading process. Ensure that all required data sources are available and responsive. Optimize the dashboard queries to reduce loading time. Provide a fallback mechanism to display a basic version of the dashboard if the full version fails to load.",

    "Users report that the application's notification system is not working. They are not receiving important alerts and updates, which affects their ability to stay informed about relevant events.":
    "Verify that the notification service is running and properly configured. Check for any issues in the notification delivery system, such as connectivity problems or message queue failures. Ensure that user preferences for notifications are correctly saved and respected. Implement logging to track notification delivery and identify any failures.",

    "The application's file download feature is not functioning correctly. Users are unable to download files, and they receive an error message indicating that the download failed. This issue is impacting their ability to access necessary documents and resources.":
    "Review the file download process and ensure that files are being served correctly from the server. Check the server logs for any errors related to file downloads. Verify that file permissions are set correctly and that files are accessible. Implement error handling to provide clear feedback to users if a download fails.",

    "Users are experiencing problems with the application's data import feature. When attempting to import data from a CSV file, the process fails with an error message, and no data is imported. This issue prevents users from updating their records efficiently.":
    "Check the data import process and ensure that the CSV parser is correctly configured. Validate the CSV file format to ensure it meets the application's requirements. Review the error logs to identify specific issues during the import process. Provide detailed error messages to users to help them correct any issues with their CSV files."
}

In [4]:
descriptions = list(proxy_dataset.keys())
solutions = list(proxy_dataset.values())

In [5]:
from process import data_preprocess,get_bert_embeddings

In [9]:
processed_tokens= data_preprocess()
print("Processed tokens:")
print(processed_tokens)

embeddings = get_bert_embeddings(processed_tokens)
print("BERT Embeddings shape:") #print its shape
print(embeddings)

Processed tokens:
[('hello', 'NN')]


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

ValueError: Found unexpected instance while processing input tensors for keras functional model. Expecting KerasTensor which is from tf.keras.Input() or output from keras layer call(). Got: [[ 101 7592  102 1050 2078  102]]